In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

If we want to have **reproducible results**, the environment variable `PYTHONHASHSEED` has to be set to a fixed value, for example to `0`.
Below we check that this environment is set so that results are reproducible.
In order to set this variable we have to use the following sequence of commands in the anaconda shell.  
```
conda activate ai
conda env config vars set PYTHONHASHSEED=0
conda activate ai
```
It is necessary to reactivate the environment `ai` for the setting to take effect.

In [2]:
%load_ext nb_mypy

Version 1.0.5


In [3]:
from typing import TypeVar

In [4]:
Value      = TypeVar('Value')
Element    = TypeVar('Element')
Variable   = str
Formula    = str
CSP        = tuple[set[Variable] | list[Variable], set[Value], set[Formula]]
ACSP       = tuple[set[Variable] | list[Variable], set[Value], list[tuple[Formula, set[Variable]]]]
Assignment = dict[Variable, Value]

# Consistency Checking

## Utility Functions

In [5]:
import ast

The function `collect_variables(expr)` takes a string `expr` that can be interpreted as a Python expression as input and collects all variables occurring in `expr`.  It takes care to eliminate the function symbols from the names returned by `extract_variables`.

In [6]:
def collect_variables(expression: Formula) -> frozenset[Variable]: 
    tree      = ast.parse(expression)
    Variables = { node.id for node in ast.walk(tree) 
                          if  isinstance(node, ast.Name) 
                          if node.id not in dir(__builtins__)
                }
    return frozenset(Variables)

The function `arb(S)` takes a set `S` as input and returns an arbitrary element from 
this set.

In [7]:
def arb(S: set[Element] | frozenset[Element]) -> Element:
    for x in S:
        return x
    assert False, 'arb called with empty set'

Backtracking is simulated by raising the `Backtrack` exception.  We define this new class of exceptions so that we can distinguish `Backtrack` exceptions from ordinary exceptions.  This is done by creating a new, empty class that is derived from the class `Exception`.  

In [8]:
class Backtrack(Exception):
    pass

Given a list of sets `L`, the function `union(L)` returns the set of all elements occurring in some set $S$ that is itself a member of the list `L`, i.e. we have
$$ \texttt{union}(L) = \{ x \mid \exists S \in L : x \in S \}. $$ 
A different way to write this equation is as follows:
$$ \texttt{union}([S_1, S_2 \cdots, S_n]) = S_1 \cup S_2 \cup \cdots \cup S_n. $$ 

In [9]:
def union(L: list[frozenset[Element]]) -> set[Element]:
    return { x for S in L
               for x in S
           }

In [10]:
union([ frozenset({1, 2}), frozenset({3, 4}), frozenset({1, 5}) ])

{1, 2, 3, 4, 5}

## A Constraint Propagation Solver with Consistency Maintenance

The following forward declarations are needed later.

In [11]:
def variables_2_formulas(Constraints: set[tuple[Formula, frozenset[Variable]]]) -> dict[Variable, set[Formula]]:
    return None # type: ignore

In [12]:
def enforce_consistency(ValuesPerVar: dict[Variable, set[Value]], 
                        Var2Formulas: dict[Variable, set[Formula]], 
                        Annotated:    dict[Formula, frozenset[Variable]], 
                        Connected:    dict[Variable, set[Variable]]) -> None:
    return None

In [13]:
def all_assignments(Variables:    set[Variable], 
                    ValuesPerVar: dict[Variable, set[Value]]) -> list[Assignment]:
    return None # type: ignore 

In [14]:
def extend(A: Assignment, x: Variable, v: Value) -> Assignment:
    return None # type: ignore

In [15]:
def exists_values(var:          Variable, 
                  val:          Value, 
                  f:            Formula, 
                  Vars:         frozenset[Variable], 
                  ValuesPerVar: dict[Variable, set[Value]]) -> bool:
    return False  # dummy value

In [16]:
def solve_unary(f: Formula, x: Variable, Values: set[Value]) -> set[Value]:
    return None # type: ignore

In [17]:
def backtrack_search(Assgnmnt:     Assignment, 
                     ValuesPerVar: dict[Variable, set[Value]], 
                     Constraints:  set[tuple[Formula, frozenset[Variable]]]
                    ) -> Assignment | None:
    return None

In [18]:
def most_constrained_variable(Assgnmnt:     Assignment, 
                              ValuesPerVar: dict[Variable, set[Value]]) -> Variable:
    return None # type: ignore

In [19]:
def propagate(x:            Variable, 
              v:            Value, 
              Assgnmnt:     Assignment, 
              Constraints:  set[tuple[Formula, frozenset[Variable]]], 
              ValuesPerVar: dict[Variable, set[Value]]) -> dict[Variable, set[Value]]:
    return None # type: ignore

The procedure `solve(P, check_consistency)` takes three arguments:
* `P` is a *constraint satisfaction problem*.

  Here `P` is a triple of the form 
  $$ \mathcal{P} = \langle \mathtt{Variables}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
  where 
  - $\mathtt{Variables}$ is a set of strings which serve as *variables*,
  - $\mathtt{Values}$ is a set of *values* that can be assigned 
    to the variables in the set $\mathtt{Variables}$.
  - $\mathtt{Constraints}$ is a set of *formulas* from first order logic.  
    Each of these formulas is  called a *constraint* of $\mathcal{P}$.
* `check_consistency` is a Boolean flag.  If this flag is `True`, then *consistency maintenance* is used as a preprocessing step.

Before trying to solve the given CSP, `solve` checks whether the set of variables occurring in
the constraints is the same as the set `Variables`.  If this is not the case, then this is most likely due to
a spelling error and a warning message is printed.  Then, the function `solve` converts the CSP `P` into an *augmented CSP* where every constraint $f$ is annotated with the variables occurring in $f$.  Furthermore, the function `solve` maintains the following data structures:

- `VarsInConstrs` is the set of all variables occurring in any constraint.
- `ValuesPerVar` is a dictionary mapping variables to sets of values.  For every variable `x` occurring in a constraint of `P`, the expression `ValuesPerVar[x]` is the set of values that can be used to instantiate the variable `x`.  Initially, `ValuesPerVar[x]` is set to `Values`, but as the search for a solution proceeds, the sets `ValuesPerVar[x]` are reduced by removing any values that cannot be part of a solution.
- `Annotated` is a dictionary.  For every constraint `f` we have that `Annotated[f]` is the set of all variables occurring in `f`.
- `UnaryConstrs` is a set of pairs of the form `(f, V)` where `f` is a constraint containing only a single variable and `V` is the set containing just this variable.
- `OtherConstrs` is a set of pairs of the form `(f, V)` where `f` is a constraint containing more than one variable and `V` is the set of all variables occurring in `f`.
- `Connected` is a dictionary mapping variables to sets of variables.  If `x` is a variable, then  `Connected[x]` is the set of those variables `y` such that there is a constraint `f` that mentions both the variable `x` and the variable `y`.
- `Var2Formulas` is a dictionary mapping variables to sets of formulas.  For every variable `x`, `Var2Formulas[x]` is the set of all those non-unary constraints `f` such that `x` occurs in `f`.

The unary constraints are immediately solved.  After that, the function `enforce_consistency` performs 
*consistency maintenance*:  
Formally, we define that a value $v$ is *consistent* for a variable $x$ with respect to a constraint $f$
iff the partial assignment $\{ x \mapsto v \}$ can be extended to an assignment $A$ satisfying the constraint $f$,
i.e. for every variable $\texttt{y}_i$ occurring in `f` there is a value $w_i \in \texttt{ValuesPerVar}[y]$ such that  
$$ \texttt{eval}\bigl(f, \{ x \mapsto v, y_1 \mapsto w_1, \cdots, y_n \mapsto w_n\}\bigr) = \texttt{True}. $$
The call to `enforce_consistency` shrinks the sets `ValuesPerVars[x]` until all values in `ValuesPerVars[x]`
are consistent with respect to all constraints.

Finally, `backtrack_search` is called to solve the remaining constraint satisfaction problem by the means of both *backtracking* and
*constraint propagation*.  Furthermore, the *most constrained variable* heuristic is used.

In [20]:
def solve(P: CSP, check_consistency: bool=True) -> Assignment | None:
    Variables, Values, Constraints = P
    Variables      = set(Variables)
    VarsInConstrs  = union([ collect_variables(f) for f in Constraints ])
    MisspelledVars = (VarsInConstrs - Variables) | (Variables - VarsInConstrs)
    if MisspelledVars:
        print("Did you misspell any of the following Variables?")
        for v in MisspelledVars:
            print(v)
    ValuesPerVar = { x: Values.copy() for x in Variables }
    Annotated    = { f: collect_variables(f) for f in Constraints }
    UnaryConstrs = { (f, V) for f, V in Annotated.items() if len(V) == 1 }
    OtherConstrs = { (f, V) for f, V in Annotated.items() if len(V) >= 2 }
    Connected    = {}
    Var2Formulas = variables_2_formulas(OtherConstrs)
    for x in Variables:
        Connected[x] = union([ V for _, V in Annotated.items() if x in V ]) - { x }
    try:
        for f, V in UnaryConstrs:
            var: str          = arb(V)
            ValuesPerVar[var] = solve_unary(f, var, ValuesPerVar[var])
            print(f'ValuesPerVar[{var}] = {ValuesPerVar[var]}')
        if check_consistency:
            enforce_consistency(ValuesPerVar, Var2Formulas, Annotated, Connected)
            for x, Values in ValuesPerVar.items():
                print(f'{x}: {Values}')
        return backtrack_search({}, ValuesPerVar, OtherConstrs)
    except Backtrack:
        return None

The function `variables_2_formulas` takes a set of *annotated constraints* as input.  An *annotated constraint* is a pair of the form
`(f, V)` where `f` is a formula and `V` is the set of variables occurring in `f`.  The function returns
a dictionary that maps every variable `x` to the set of those constraints `f` such that `x` occurs in `f`.

In [21]:
def variables_2_formulas(Constraints: set[tuple[Formula, frozenset[Variable]]]) -> dict[Variable, set[Formula]]:
    Dictionary: dict[str, set[str]] = {}
    for f, Vars in Constraints:
        for x in Vars: 
            if x in Dictionary: # Dictionary[x] is already defined
                Dictionary[x] |= { f }
            else:               # Dictionary[x] is not yet defined
                Dictionary[x]  = { f }
    return Dictionary

The function `enforce_consistency` takes 4 arguments:
- `ValuesPerVar` is a dictionary.  For every variable `x` we have that `ValuesPerVar[x]` is the set of values that can be substituted for `x`.
- `Var2Formulas` is a dictionary.  For every variable `x` we have that `Var2Formulas[x]` is the set of those formulas that mention the variable `x`.
- `Annotated` is a dictionary.  For every constraint `f`, `Annotated[f]` is the set of variables occurring in `f`.
- `Connected` is a dictionary.  For every variable `x` we have that `Connected[x]` is the set of those variables `y` that are *directly connected* with the variable `x`.  Two variables `x` and `y` are *directly connected* if there is a constraint `F` such that both `x` and `y` occur in `F`.  In this case, `F` is *connecting* `x` and `y`.

The function `enforce_consistency` shrinks the sets `ValuesPerVar[x]` such that the values in `ValuesPerVar[x]` are consistent for `x` for all constraints.

In [22]:
def enforce_consistency(ValuesPerVar: dict[Variable, set[Value]], 
                        Var2Formulas: dict[Variable, set[Formula]], 
                        Annotated:    dict[Formula, frozenset[Variable]], 
                        Connected:    dict[Variable, set[Variable]]) -> None:
    UncheckedVars = set(Var2Formulas.keys())
    while UncheckedVars:
        variable    = UncheckedVars.pop()
        RemovedVals = set()
        for f in Var2Formulas[variable]:
            OtherVars = Annotated[f] - { variable }
            for value in ValuesPerVar[variable]:
                if not exists_values(variable, value, f, OtherVars, ValuesPerVar):
                    RemovedVals   |= { value }
                    UncheckedVars |= Connected[variable]
        ValuesPerVar[variable] -= RemovedVals
        if len(ValuesPerVar[variable]) == 0: # the problem is unsolvable
            raise Backtrack()

The procedure `exists_values` takes five arguments:
- `var` is a variable, 
- `val` is a value val, 
- `f`   is a constraint,
- `Vars` is the set Vars of those variables in f that are different from `var`, and
- `ValuesPerVar` is a dictionary.  For every variable `x` we have that `ValuesPerVar[x]` is the set of those values that still may be tried for `x`.

The function checks whether there is a value for `var` such that the other variables occurring in the constraint `f` can be set to values such that the constraint `f` is satisfied.

In [23]:
def exists_values(var:          Variable, 
                  val:          Value, 
                  f:            Formula, 
                  Vars:         frozenset[Variable], 
                  ValuesPerVar: dict[Variable, set[Value]]) -> bool:
    Assignments = all_assignments(set(Vars), ValuesPerVar)
    return any(eval(f, extend(A, var, val)) for A in Assignments)

The function `extend` takes three arguments:
- `A` is a dictionary,
- `x` is a variable such that `A[x]`is not yet defined,
- `v` is some value.

It returns a new dictionary `B` such that `B[x] = v` and `B[y] = A[y]` for all `y != x`.

In [24]:
def extend(A: Assignment, x: Variable, v: Value) -> Assignment:
    B = A.copy()
    B[x] = v
    return B

The function `all_assignments` returns the list of all possible assignments for the variables in the set Vars.
For every variable `x`, the values for `x` are taken from `ValuesPerVar[x]`.

**Nota Bene:** If there are $n$ variables and $m$ values for each variable, then there are $m^n$ possible assignments. Hence the size of the returned lists grows exponentially with the number of variables.

In [25]:
def all_assignments(Variables:    set[Variable], 
                    ValuesPerVar: dict[Variable, set[Value]]) -> list[Assignment]:
    if not Variables:
        return [ {} ]  # list containing empty assignment
    var         = Variables.pop()
    Values      = ValuesPerVar[var]
    Assignments = all_assignments(Variables, ValuesPerVar)
    return [ extend(A, var, val) for A in Assignments 
                                 for val in ValuesPerVar[var]
           ]

In [26]:
ValuesPerVar = { 'x': {1, 2}, 'y': {2, 3} }
Variables    = { 'x', 'y' }
all_assignments(Variables, ValuesPerVar)

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 2}, {'x': 2, 'y': 3}]

The function `solve_unary` takes a unary constraint `f`, a variable `x` and the set of values `Values` that can be assigned to `x`.  It returns the subset of values that can be substituted for `x` such that $f[x\mapsto v]$ evaluates as `True`.

In [27]:
def solve_unary(f: Formula, x: Variable, Values: set[Value]) -> set[Value]:
    Legal = { value for value in Values if  eval(f, { x: value }) }
    if not Legal:
        raise Backtrack()
    return Legal

The function `backtrack_search` takes three arguments:
- `Assignment` is a partial variable assignment that is represented as a
   dictionary.  Initially, this assignment will be the  empty dictionary.     
   Every recursive call of `backtrack_search` adds the assignment of one 
   variable to  the given assignment. 
- `ValuesPerVar` is a dictionary.  For every variable `x`, `ValuesPerVar[x]` is the set of values that still might be assigned to `x`.
- `Constraints` is a set of pairs of the form `(f, V)` where `f` is a constraint and `V` is the set of variables occurring in `f`.

The function `backtrack_search` uses the *most constrained variable heuristic* in order to choose the next variable.  It uses the *least constraining value heuristic* to choose the value that is assigned to this variable.

In [28]:
def backtrack_search(Assgnmnt:     Assignment, 
                     ValuesPerVar: dict[Variable, set[Value]], 
                     Constraints:  set[tuple[Formula, frozenset[Variable]]]
                    ) -> Assignment | None:
    if len(Assgnmnt) == len(ValuesPerVar):
        return Assgnmnt
    x = most_constrained_variable(Assgnmnt, ValuesPerVar)
    Values = ValuesPerVar[x]
    for v in Values: 
        try:
            NewValues    = propagate(x, v, Assgnmnt, Constraints, ValuesPerVar)
            NewAssign    = Assgnmnt.copy()
            NewAssign[x] = v
            return backtrack_search(NewAssign, NewValues, Constraints)
        except Backtrack:
            continue
    raise Backtrack()

The function `most_constrained_variable` takes two parameters:
- `Assigment` is a *partial variable assignment* that assigns values to variables.  It is represented as a dictionary.
- `ValuesPerVar` is a dictionary that has variables as keys.  For every variable `x`, `ValuesPerVar[x]` is the set of values that 
  can still be assigned to the variable `x`.
  
The function returns an unassigned variable `x` such that the number of values in `ValuesPerVar[x]` is minimal among all other unassigned variables.
Hence, this variable is a *most constraint variable*.  
* In order to find this variable, the set `Unassigned` is computed.  This is a set of pairs of the form `(x, n)` 
  where `x` is a variable that is not yet assigned and `n` is the number of values that can still be assigned to `x`.
* `minSize` is the number of values that can be assigned to a most constrained variable.  

In [29]:
def most_constrained_variable(Assgnmnt:     Assignment, 
                              ValuesPerVar: dict[Variable, set[Value]]) -> Variable:
    Unassigned = { len(U): x for x, U in ValuesPerVar.items()
                             if  x not in Assgnmnt
                 }
    minSize = min(Unassigned.keys())
    return Unassigned[minSize]

The function `propagate` takes five arguments:
- `x` is a variable,
- `v` is a value that is supposed to be assigned to `x`.
- `Assignment` is a partial assignment that contains assignments for variables that are different from `x`.
- `Constraints` is a set of annotated constraints.
- `ValuesPerVar` is a dictionary assigning sets of values to all variables.  For every unassigned variable `z`,  `ValuesPerVar[z]` is the set of values that still might be assigned to `z`.

The purpose of the function  `propagate` is to compute how the sets `ValuesPerVar[z]` shrink when the value `v` is assigned to the variable `x`.  The dictionary `ValuesPerVar` with appropriately reduced sets `ValuesPerVar[z]` is returned.  In particular, the consequences of assigning the value `v` to the variable `x` are *propagated*:
If there is a constraint `f` such that `x` occurs in `f` and there is just one variable `y` left that occurs in 
`f` and that is not yet bound in `Assignment`, then the values that can still be assigned to `y` are computed
and the dictionary `ValuesDict` is updated accordingly.  If there are no values left that can be assigned to 
`y` without violating the constraint `f`, the function backtracks.

In [30]:
def propagate(x:            Variable, 
              v:            Value, 
              Assgnmnt:     Assignment, 
              Constraints:  set[tuple[Formula, frozenset[Variable]]], 
              ValuesPerVar: dict[Variable, set[Value]]) -> dict[Variable, set[Value]]:
    ValuesDict    = ValuesPerVar.copy()
    ValuesDict[x] = { v }
    BoundVars     = set(Assgnmnt.keys())
    for f, Vars in Constraints:
        if x in Vars:
            UnboundVars = Vars - BoundVars - { x }
            if len(UnboundVars) == 1:
                y = arb(UnboundVars)
                Legal = set()
                for w in ValuesDict[y]:
                    NewAssign = Assgnmnt.copy()
                    NewAssign[x] = v
                    NewAssign[y] = w
                    if eval(f, NewAssign):
                        Legal.add(w)
                if not Legal:
                    raise Backtrack()
                ValuesDict[y] = Legal
    return ValuesDict

## Solving the *Eight-Queens-Puzzle*

In [31]:
%run NQueensProblemCSP.ipynb

The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy
Variables:   ['V1', 'V2', 'V3', 'V4']
Values:      {1, 2, 3, 4}
Constraints:
             abs(V2 - V1) != 1
             abs(V3 - V1) != 2
             V1 != V4
             abs(V4 - V3) != 1
             V2 != V3
             abs(V4 - V1) != 3
             V2 != V4
             V1 != V3
             abs(V3 - V2) != 1
             abs(V4 - V2) != 2
             V1 != V2
             V3 != V4
  Cloning https://github.com/reclinarka/chess-problem-visuals to /private/var/folders/7q/vnj0c8xx4njcz6k2886gd6680000gn/T/pip-req-build-m07hyau0
  Running command git clone --filter=blob:none --quiet https://github.com/reclinarka/chess-problem-visuals /private/var/folders/7q/vnj0c8xx4njcz6k2886gd6680000gn/T/pip-req-build-m07hyau0
  Resolved https://github.com/reclinarka/chess-problem-visuals to commit 764a29b376fe9dd3cbb2623ce8740f73c6711fa4
  Preparing metadata (setup.py) ... done


In [32]:
%unload_ext nb_mypy

In [33]:
P = create_csp(8)

The consistency solver takes about 23 milliseconds on my desktop to solve the eight queens puzzle.  Hence, for the eight queens puzzle, consistency maintenance does not help.

In [34]:
%%time
Solution = solve(P, check_consistency=True)
print(f'Solution = {Solution}')

V3: {1, 2, 3, 4, 5, 6, 7, 8}
V5: {1, 2, 3, 4, 5, 6, 7, 8}
V4: {1, 2, 3, 4, 5, 6, 7, 8}
V8: {1, 2, 3, 4, 5, 6, 7, 8}
V1: {1, 2, 3, 4, 5, 6, 7, 8}
V2: {1, 2, 3, 4, 5, 6, 7, 8}
V7: {1, 2, 3, 4, 5, 6, 7, 8}
V6: {1, 2, 3, 4, 5, 6, 7, 8}
Solution = {'V6': 1, 'V7': 3, 'V2': 2, 'V8': 6, 'V3': 8, 'V4': 5, 'V5': 7, 'V1': 4}
CPU times: user 28.7 ms, sys: 1.88 ms, total: 30.5 ms
Wall time: 29.4 ms


We can see that consistency maintenance was not able to reduce the set of values for any of the variables.  Hence, for the 8-queens-puzzle it does not help.

In [35]:
show_solution(Solution, 8)

In [36]:
%%time
Solution = solve(P, check_consistency=False)
print(f'Solution = {Solution}')

Solution = {'V6': 1, 'V7': 3, 'V2': 2, 'V8': 6, 'V3': 8, 'V4': 5, 'V5': 7, 'V1': 4}
CPU times: user 18.6 ms, sys: 2.73 ms, total: 21.4 ms
Wall time: 18.8 ms


In [37]:
P = create_csp(32)

The 32-queens-problem can be solved in 1 second if we use consistence maintenance.
Again, we see that consistency maintenance is not usefull for the n-queens-puzzle.

In [38]:
%%time
Solution = solve(P, check_consistency=True)
show_solution(Solution, 32, "60%")

V3: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V10: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V25: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V31: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V19: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V30: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V6: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V18: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
V3

## Solving the *Zebra Puzzle*

In [39]:
%run Zebra.ipynb

Version 1.0.5


In [40]:
%unload_ext nb_mypy

In [41]:
zebra = zebra_csp()

The consistency solver takes about 11 milliseconds to solve the *zebra puzzle* without consistency maintenance.

In [42]:
%%time
Solution = solve(zebra, check_consistency=False)

ValuesPerVar[Norwegian] = {1}
ValuesPerVar[Milk] = {3}
CPU times: user 21.6 ms, sys: 2.87 ms, total: 24.4 ms
Wall time: 22 ms


In [43]:
show_solution(Solution)

House,Nationality,Drink,Animal,Brand,Colour
1,Norwegian,Water,Fox,Kools,Yellow
2,Ukrainian,Tea,Horse,Chesterfields,Blue
3,English,Milk,Snails,OldGold,Red
4,Spanish,OrangeJuice,Dog,LuckyStrike,Ivory
5,Japanese,Coffee,Zebra,Parliaments,Green


For the *Zebra puzzle*, *consistency maintenance* is able to decrease the set of values which have to be 
tried for the different variables, but it does not decrease the running time.

In [44]:
%%time
Solution = solve(zebra, check_consistency=True)
show_solution(Solution)

ValuesPerVar[Norwegian] = {1}
ValuesPerVar[Milk] = {3}
Green: {4, 5}
Zebra: {1, 2, 3, 4, 5}
Snails: {1, 2, 3, 4, 5}
Horse: {2, 3, 4, 5}
OrangeJuice: {1, 2, 4, 5}
Parliaments: {2, 3, 4, 5}
OldGold: {1, 2, 3, 4, 5}
LuckyStrike: {1, 2, 4, 5}
Japanese: {2, 3, 4, 5}
Fox: {1, 2, 3, 4, 5}
Red: {3, 4, 5}
Dog: {2, 3, 4, 5}
Yellow: {1, 3, 4, 5}
Norwegian: {1}
Water: {1, 2, 4, 5}
Ukrainian: {2, 4, 5}
Kools: {1, 3, 4, 5}
English: {3, 4, 5}
Blue: {2}
Milk: {3}
Spanish: {2, 3, 4, 5}
Chesterfields: {1, 2, 3, 4, 5}
Ivory: {3, 4}
Tea: {2, 4, 5}
Coffee: {4, 5}


House,Nationality,Drink,Animal,Brand,Colour
1,Norwegian,Water,Fox,Kools,Yellow
2,Ukrainian,Tea,Horse,Chesterfields,Blue
3,English,Milk,Snails,OldGold,Red
4,Spanish,OrangeJuice,Dog,LuckyStrike,Ivory
5,Japanese,Coffee,Zebra,Parliaments,Green


CPU times: user 28.9 ms, sys: 2.43 ms, total: 31.3 ms
Wall time: 29.5 ms


## Solving a Sudoku Puzzle

In [45]:
%run Sudoku.ipynb

  Cloning https://github.com/reclinarka/problem_visuals to /private/var/folders/7q/vnj0c8xx4njcz6k2886gd6680000gn/T/pip-req-build-zafn45mc
  Running command git clone --filter=blob:none --quiet https://github.com/reclinarka/problem_visuals /private/var/folders/7q/vnj0c8xx4njcz6k2886gd6680000gn/T/pip-req-build-zafn45mc
  Resolved https://github.com/reclinarka/problem_visuals to commit 5a7abd2897400e33220fd32be23a2e4f70661f21
  Preparing metadata (setup.py) ... done


In [46]:
%unload_ext nb_mypy

The nb_mypy extension is not loaded.


In [47]:
csp = sudoku_csp()

In [48]:
%%time
Solution = solve(csp, check_consistency=False)

ValuesPerVar[V39] = {3}
ValuesPerVar[V88] = {4}
ValuesPerVar[V97] = {5}
ValuesPerVar[V28] = {9}
ValuesPerVar[V82] = {2}
ValuesPerVar[V76] = {9}
ValuesPerVar[V47] = {2}
ValuesPerVar[V38] = {8}
ValuesPerVar[V83] = {1}
ValuesPerVar[V19] = {7}
ValuesPerVar[V13] = {9}
ValuesPerVar[V12] = {3}
ValuesPerVar[V29] = {2}
ValuesPerVar[V55] = {4}
ValuesPerVar[V27] = {4}
ValuesPerVar[V74] = {2}
ValuesPerVar[V48] = {7}
ValuesPerVar[V36] = {5}
ValuesPerVar[V57] = {8}
ValuesPerVar[V46] = {3}
ValuesPerVar[V62] = {6}
ValuesPerVar[V91] = {7}
ValuesPerVar[V44] = {6}
ValuesPerVar[V79] = {1}
ValuesPerVar[V24] = {7}
ValuesPerVar[V35] = {6}
ValuesPerVar[V73] = {5}
ValuesPerVar[V61] = {5}
CPU times: user 103 ms, sys: 3.21 ms, total: 106 ms
Wall time: 106 ms


In [49]:
show_solution(Solution)

In [50]:
%%time
Solution = solve(csp, check_consistency=True)

ValuesPerVar[V39] = {3}
ValuesPerVar[V88] = {4}
ValuesPerVar[V97] = {5}
ValuesPerVar[V28] = {9}
ValuesPerVar[V82] = {2}
ValuesPerVar[V76] = {9}
ValuesPerVar[V47] = {2}
ValuesPerVar[V38] = {8}
ValuesPerVar[V83] = {1}
ValuesPerVar[V19] = {7}
ValuesPerVar[V13] = {9}
ValuesPerVar[V12] = {3}
ValuesPerVar[V29] = {2}
ValuesPerVar[V55] = {4}
ValuesPerVar[V27] = {4}
ValuesPerVar[V74] = {2}
ValuesPerVar[V48] = {7}
ValuesPerVar[V36] = {5}
ValuesPerVar[V57] = {8}
ValuesPerVar[V46] = {3}
ValuesPerVar[V62] = {6}
ValuesPerVar[V91] = {7}
ValuesPerVar[V44] = {6}
ValuesPerVar[V79] = {1}
ValuesPerVar[V24] = {7}
ValuesPerVar[V35] = {6}
ValuesPerVar[V73] = {5}
ValuesPerVar[V61] = {5}
V66: {1, 2, 7, 8}
V78: {3, 6}
V25: {1, 3, 8}
V31: {2, 4}
V75: {3, 7, 8}
V58: {1, 3, 6}
V46: {3}
V88: {4}
V19: {7}
V41: {1, 4, 8, 9}
V99: {6, 8, 9}
V51: {1, 2, 3, 9}
V77: {3, 7}
V98: {2, 3, 6}
V18: {5}
V83: {1}
V65: {1, 2, 7, 8, 9}
V97: {5}
V69: {4, 9}
V92: {4, 8, 9}
V12: {3}
V35: {6}
V72: {4, 8}
V79: {1}
V14: {1, 4, 8}
V54: {1

In [51]:
%%time
Solution = solve(csp, check_consistency=False)

ValuesPerVar[V39] = {3}
ValuesPerVar[V88] = {4}
ValuesPerVar[V97] = {5}
ValuesPerVar[V28] = {9}
ValuesPerVar[V82] = {2}
ValuesPerVar[V76] = {9}
ValuesPerVar[V47] = {2}
ValuesPerVar[V38] = {8}
ValuesPerVar[V83] = {1}
ValuesPerVar[V19] = {7}
ValuesPerVar[V13] = {9}
ValuesPerVar[V12] = {3}
ValuesPerVar[V29] = {2}
ValuesPerVar[V55] = {4}
ValuesPerVar[V27] = {4}
ValuesPerVar[V74] = {2}
ValuesPerVar[V48] = {7}
ValuesPerVar[V36] = {5}
ValuesPerVar[V57] = {8}
ValuesPerVar[V46] = {3}
ValuesPerVar[V62] = {6}
ValuesPerVar[V91] = {7}
ValuesPerVar[V44] = {6}
ValuesPerVar[V79] = {1}
ValuesPerVar[V24] = {7}
ValuesPerVar[V35] = {6}
ValuesPerVar[V73] = {5}
ValuesPerVar[V61] = {5}
CPU times: user 95 ms, sys: 1.98 ms, total: 97 ms
Wall time: 97.2 ms


## Solving a Crypto-Arithmetic Puzzle

In [52]:
%run CryptoArithmetic.ipynb

Version 1.0.5


In [53]:
%unload_ext nb_mypy

In [54]:
csp = crypto_csp()

With *consistency checking* the time to solve the crypto arithmetic puzzle is reduced to less than 50 milliseconds.

In [55]:
%%time
Solution = solve(csp, check_consistency=True)

ValuesPerVar[S] = {1, 2, 3, 4, 5, 6, 7, 8, 9}
ValuesPerVar[M] = {1, 2, 3, 4, 5, 6, 7, 8, 9}
S: {8, 9}
E: {0, 2, 3, 4, 5, 6, 7, 8, 9}
N: {0, 2, 3, 4, 5, 6, 7, 8, 9}
M: {1}
Y: {0, 2, 3, 4, 5, 6, 7, 8, 9}
C2: {0, 1}
D: {0, 2, 3, 4, 5, 6, 7, 8, 9}
C1: {0, 1}
R: {0, 2, 3, 4, 5, 6, 7, 8, 9}
O: {0, 9}
C3: {0, 1}
CPU times: user 144 ms, sys: 2.3 ms, total: 146 ms
Wall time: 146 ms


In [56]:
show_solution(Solution)

S = 9
E = 5
N = 6
M = 1
Y = 2
D = 7
R = 8
O = 0

The solution of

    S E N D
  + M O R E
  ---------
  M O N E Y

is as follows

    9 5 6 7
  + 1 0 8 5
  1 0 6 5 2


Without consistency checking, the problem takes 622 milliseconds.

In [57]:
%%time
Solution = solve(csp, check_consistency=False)

ValuesPerVar[S] = {1, 2, 3, 4, 5, 6, 7, 8, 9}
ValuesPerVar[M] = {1, 2, 3, 4, 5, 6, 7, 8, 9}
CPU times: user 614 ms, sys: 2.25 ms, total: 616 ms
Wall time: 619 ms


In [ ]:
csp = crypto_csp_hard()
csp

In [ ]:
%%time
Solution = solve(csp, check_consistency=True)

For the hard version of the crypto-arithmetic puzzle, *consistency maintenance* decreases the total running time considerably.

In [ ]:
%%time
Solution = solve(csp, check_consistency=False)